# Script to record data from RAV4 Using Panda/Giraffe

#### Before you start working with panda make sure you have executed following command in the terminal.
`sudo -i`

`echo 'SUBSYSTEMS=="usb", ATTR{idVendor}=="bbaa", ATTR{idProduct}=="ddcc", MODE:="0666"' > /etc/udev/rules.d/11-panda.rules`

`exit`


## Install Panda from clone repository.

In [ ]:
# You don't to run it if you have already run it. By default I will comment it out!
!pip uninstall pandacan -y
!cd ..; python setup.py install

## Install other necessary packages

In [ ]:
# You don't to run it if you have already run it. By default I will comment it out!
!pip install cantools tqdm libusb1 pyserial bitstring

## Import  Packages

In [ ]:
import binascii
import bitstring
import time
import datetime
import serial
import csv
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd # Note that this is not commai Panda, but Database Pandas
import cantools 
import matplotlib.animation as animation
from matplotlib import style
import uuid

## Connect to Panda

In [ ]:
from panda import Panda #Import Comma AI Panda
__vehicleName__ = 'Rav4'
panda = Panda()

In [ ]:
#from panda import PandaWifiStreaming
#panda = PandaWifiStreaming(ip="192.168.0.11")

In [ ]:
can_recv = panda.can_recv(14096) 

In [ ]:
len(can_recv)

## Record data in infinite loop

In [ ]:
len(can_recv)

### To Stope recrod, go to Kernel -> Interrupt

In [ ]:
__vehicleName__ = 'Rav4'
# Create a Unique File Name
#unique_filename = str(uuid.uuid4())
currTime = str(time.time())
dt_object = datetime.datetime.fromtimestamp(time.time())
dt = dt_object.strftime('%Y-%m-%d-%H-%M-%S-%f')
fileName = dt + '_' + '_CAN_Message_'+__vehicleName__+'.csv'
rf_PANDA = open(fileName, 'a')
print('Writing: '+fileName)
csvwriter_PANDA = csv.writer(rf_PANDA)
csvwriter_PANDA.writerow(['Time','Bus', 'MessageID', 'Message', 'MessageLength'])

counter = 0
while True:
    #counter = counter + 1
    can_recv = panda.can_recv() # collects packages, 256 at a time
    #print(can_recv)
    currTime = time.time() # Records time of collection
    for messageID, _, dat, bus  in can_recv:
        # Be careful changing this, can be picky:
        #if address == 467:
        #    print(can_recv)
        #    print('Bus = ',  bus)
        #    print('\n')
        #    print('Message ID = ', messageID)
        #    print('\n')
        #    print('Message = ', dat)
        #    print('\n')
        csvwriter_PANDA.writerow(([str(currTime), str(bus), str((messageID)), str(binascii.hexlify(dat).decode('utf-8')), len(dat)]))
        #print('Addres: '+str((address)), end='\r')

## Before you remove USB from Lapop, make sure to run `panda.close()`

In [ ]:
panda.close()